## DNA sequence statistics (1)

### R packages/libraries

Many people have written functions for doing analysis in R. The functions are put in R **packages** and **libraries** that must be installed or loaded so the functions can be used.

### Getting started

1. In the "my_notebooks/week01" folder, open a notebook from this URL https://raw.githubusercontent.com/hlab1/teaching-fb2025/main/week01/w01p2_dnaseq1.ipynb.
2. Clear all outputs by "Kernel"->"Restart Kernel and Clear All Outputs".

At any time if you want to stop the notebook, remember to "Save and create checkpoint" your notebook before doing "Close and Shutdown Notebook". And "Stop My Server" when you are done.

In [1]:
library("rentrez")
library("seqinr")
library("XML")

### Retriving genome sequence data via NCBI website

We will now go to the NCBI website to retrieve the genome sequence of the Dengue virus type 1 DEN-1, which has the accession number NC_001477.
1. Go to http://www.ncbi.nlm.nih.gov/.
2. Enter "NC_001477" in the search box "All Databases".
3. On the results page, click on the "Nucleotide" entry under "Genomes".
4. Take a look at the results page, and identify some **attributes** of this **record**.
5. Click on the "FASTA" link. This shows the sequencce of the genome in **FASTA** format.

### Retrieving genome sequence data in R using the rentrez package

We can also retrieve the genome sequence, put it in the memory of R and perform computational analysis.

First we make our own function `getncbiseq` that allows us to get a sequence using its accession number.  This function is in the cell below.

Pay attention to the first line, where we define the `function name` and the `arguments`, and don't worry about what's in the function.

In [2]:
getncbiseq <- function(db,accession) {
    require("rentrez")
    require("seqinr")
    require("XML")
    res_xml <- entrez_fetch(db,accession,rettype="fasta",retmode="xml",parsed=TRUE)
    res_list <- xmlToList(res_xml)
    res_seq <- res_list[[1]][['TSeq_sequence']]
    res_vec <- tolower(s2c(res_seq))
    return(res_vec)
}

We can now call this function with to query the "nuccore" database with accession number "NC_001477" as arguments.

In [3]:
dengueseq <- getncbiseq("nuccore","NC_001477")

Aside: a string without quotes is a variable name and a string inside quotes is a string.  So the `dengueseq` above is the name of a variable now holding the sequence of the Dengue genome. "NC_001477" is a string with value "NC_001477"

The variable `dengueseq` now is a vector of single characters that contain the nucleotide sequence. Each element in the vector is one nucleotide. We can use single square bracket as before to get the nucleotides at specific positions.

In [4]:
dengueseq[1:50]

[1] "a" "g" "t" "t" "g" "t" "t" "a" "g" "t" "c" "t" "a" "c" "g" "t" "g" "g" "a"
[20] "c" "c" "g" "a" "c" "a" "a" "g" "a" "a" "c" "a" "g" "t" "t" "t" "c" "g" "a"
[39] "a" "t" "c" "g" "g" "a" "a" "g" "c" "t" "t" "g"

We can write this sequence to a FASTA file.  Check back at your "my_notebooks/week01" folder.  You should now see the "den1.fasta" file.

In [5]:
write.fasta(names="DEN-1", sequences=dengueseq, file.out="den1.fasta")

Read the sequence file back into R.

In [6]:
dengueseq_fromfile <- read.fasta(file = "den1.fasta")

Now what is in this new `dengue_fromfile` object?

In [7]:
dengueseq_fromfile

$`DEN-1`
    [1] "a" "g" "t" "t" "g" "t" "t" "a" "g" "t" "c" "t" "a" "c" "g" "t" "g" "g"
   [19] "a" "c" "c" "g" "a" "c" "a" "a" "g" "a" "a" "c" "a" "g" "t" "t" "t" "c"
   [37] "g" "a" "a" "t" "c" "g" "g" "a" "a" "g" "c" "t" "t" "g" "c" "t" "t" "a"
   [55] "a" "c" "g" "t" "a" "g" "t" "t" "c" "t" "a" "a" "c" "a" "g" "t" "t" "t"
   [73] "t" "t" "t" "a" "t" "t" "a" "g" "a" "g" "a" "g" "c" "a" "g" "a" "t" "c"
   [91] "t" "c" "t" "g" "a" "t" "g" "a" "a" "c" "a" "a" "c" "c" "a" "a" "c" "g"
  [109] "g" "a" "a" "a" "a" "a" "g" "a" "c" "g" "g" "g" "t" "c" "g" "a" "c" "c"
  [127] "g" "t" "c" "t" "t" "t" "c" "a" "a" "t" "a" "t" "g" "c" "t" "g" "a" "a"
  [145] "a" "c" "g" "c" "g" "c" "g" "a" "g" "a" "a" "a" "c" "c" "g" "c" "g" "t"
  [163] "g" "t" "c" "a" "a" "c" "t" "g" "t" "t" "t" "c" "a" "c" "a" "g" "t" "t"
  [181] "g" "g" "c" "g" "a" "a" "g" "a" "g" "a" "t" "t" "c" "t" "c" "a" "a" "a"
  [199] "a" "g" "g" "a" "t" "t" "g" "c" "t" "t" "t" "c" "a" "g" "g" "c" "c" "a"
  [217] "a" "g" "g" "a" "c" "c"

It is a list with one element, whose name is "DEN-1" and value is a vector of the sequence.  Let's get this vector.

In [8]:
dengueseq_fromfile_seq <-dengueseq_fromfile[['DEN-1']]

### Calculating some statistics of this sequence.

Length and base composition:

In [9]:
length(dengueseq_fromfile_seq)

[1] 10735

In [10]:
table(dengueseq_fromfile_seq)

dengueseq_fromfile_seq
   a    c    g    t 
3426 2240 2770 2299 

What is the GC content?

In [11]:
(2240+2770)/(3426+2240+2770+2299)

[1] 0.4666977

How does this result compared to the result computed using the built-in `GC` function?

In [12]:
GC(dengueseq_fromfile_seq)

[1] 0.4666977

Frequencies of DNA **words**:

In [13]:
count(dengueseq_fromfile_seq,1)


   a    c    g    t 
3426 2240 2770 2299 

What do you think this is? Take a look at the help of the `count` function.

In [14]:
?count

count                  package:seqinr                  R Documentation

_C_o_m_p_o_s_i_t_i_o_n _o_f _d_i_m_e_r/_t_r_i_m_e_r/_e_t_c _o_l_i_g_o_m_e_r_s

_D_e_s_c_r_i_p_t_i_o_n:

     Counts the number of times dimer/trimer/etc oligomers occur in a
     sequence. Note that the oligomers are overlapping by default.

_U_s_a_g_e:

     count(seq, wordsize, start = 0, by = 1,
      freq = FALSE, alphabet = s2c("acgt"), frame = start)
     
_A_r_g_u_m_e_n_t_s:

     seq: a vector of single characters.

wordsize: an integer giving the size of word (n-mer) to count.

   start: an integer (0, 1, 2,...) giving the starting position to
          consider in the sequence. The default value 0 means that we
          start at the first nucleotide in the sequence.

      by: an integer defaulting to 1 for the window step.

    freq: if TRUE, word relative frequencies (summing to 1) are
          returned instead of counts

alphabet: a vector of single characters used to build the oligomer set.

   frame: synonymous for start

_D_e_t_a_i_l_s:

     ‘count’ counts the occurence of all words by moving a window of
     length ‘word’. The window step is controlled by the argument ‘by’.
     ‘start’ controls the starting position in the sequence for the
     count.

_V_a_l_u_e:

     This function returns a ‘table’ whose ‘dimnames’ are all the
     possible oligomers. All oligomers are returned, even if absent
     from the sequence.

_A_u_t_h_o_r(_s):

     D. Charif, J.R. Lobry with suggestions from Gabriel Valiente,
     Stefanie Hartmann and Christian Gautier

_R_e_f_e_r_e_n_c_e_s:

     ‘citation("seqinr")’

_S_e_e _A_l_s_o:

     ‘table’ for the class of the returned objet. See ‘rho’ and
     ‘zscore’ for dinucleotide statistics.

_E_x_a_m_p_l_e_s:

     a <- s2c("acgggtacggtcccatcgaa")
     ##
     ## To count dinucleotide occurrences in sequence a:
     ##
     count(a, word = 2)
     ##
     ## To count trinucleotide occurrences in sequence a, with start = 2:
     ##
     count(a, word = 3, start = 2)
     ##
     ## To count dinucleotide relative frequencies in sequence a:
     ##
     count(a, word = 2, freq = TRUE)
     ##
     ## To count dinucleotides in codon positions III-I in a coding sequence:
     ##
     alldinuclIIIpI <- s2c("NNaaNatNttNtgNgtNtcNctNtaNagNggNgcNcgNgaNacNccNcaNN")
     resIIIpI <- count(alldinuclIIIpI, word = 2, start = 2, by = 3)
     stopifnot(all( resIIIpI == 1))
     ##
     ## Simple sanity check:
     ##
     #alldinucl <- "aattgtctaggcgacca"
     #stopifnot(all(count(s2c(alldinucl), 2) == 1))
     #alldiaa <- "aaxxzxbxvxyxwxtxsxpxfxmxkxlxixhxgxexqxcxdxnxrxazzbzvzyzwztzszpzfzmzkzlzizhzgzezqzczdznz
     #rzabbvbybwbtbsbpbfbmbkblbibhbgbebqbcbdbnbrbavvyvwvtvsvpvfvmvkvlvivhvgvevqvcvdvnvrvayywytysypyfymyky
     #lyiyhygyeyqycydynyryawwtwswpwfwmwkwlwiwhwgwewqwcwdwnwrwattstptftmtktltithtgtetqtctdtntrtasspsfsmsks
     #lsishsgsesqscsdsnsrsappfpmpkplpiphpgpepqpcpdpnprpaffmfkflfifhfgfefqfcfdfnfrfammkmlmimhmgmemqmcmdmnm
     #rmakklkikhkgkekqkckdknkrkallilhlglelqlcldlnlrlaiihigieiqicidiniriahhghehqhchdhnhrhaggegqgcgdgngrgae
     #eqecedenereaqqcqdqnqrqaccdcncrcaddndrdannrnarra"
     #stopifnot(all(count(s2c(alldiaa), 2, alphabet = s2c("arndcqeghilkmfpstwyvbzx")) == 1))
     ##
     ## Example with dinucleotide count in the complete Human mitochondrion genome:
     ##
     humanMito <- read.fasta(file = system.file("sequences/humanMito.fasta", package = "seqinr"))
     ##
     ## Get the dinucleotide count:
     ##
     dinu <- count(humanMito[[1]], 2)
     ##
     ## Put the results in a 4 X 4 array:
     ##
     dinu2 <- dinu
     dim(dinu2) <- c(4, 4)
     nucl <- s2c("ACGT")
     dimnames(dinu2) <- list(paste(nucl, "-3\'", sep = ""), paste("5\'-", nucl, sep = ""))
     ##
     ## Show that CpG and GpT dinucleotides are depleted:
     ##
     mosaicplot(t(dinu2), shade = TRUE,
       main = "Dinucleotide XpY frequ

What if we add an argument, the number 2, when we call the `count` function?

In [15]:
count(dengueseq_fromfile_seq,2)


  aa   ac   ag   at   ca   cc   cg   ct   ga   gc   gg   gt   ta   tc   tg   tt 
1108  720  890  708  901  523  261  555  976  500  787  507  440  497  832  529 

You can save the results to a variable and retrieve the element that you are interested in.

In [16]:
dengueseq_table_1bp <- count(dengueseq_fromfile_seq,1)
dengue_table_2bp <- count(dengueseq_fromfile_seq,2)
dengue_table_3bp <- count(dengueseq_fromfile_seq,3)

In [17]:
dengueseq_table_1bp[["c"]]

[1] 2240

In [18]:
dengue_table_2bp[["tt"]]

[1] 529

In [19]:
dengue_table_3bp[["atg"]]

[1] 292

### Finding specific DNA words in a sequence

`count` gives you the number of occurrences of each DNA words in a sequence, for example, the number of times the start codon 'ATG' is found. What if we want to find where the 'ATG' are in the sequence?

You can use the `matchPattern` function in the *Biostrings* package.  The first argument is a pattern string, in our case 'atg'. The second argument is the sequence where the pattern will be searched. This argument needs to be a string.  Since the genome sequence `dengueseq_fromfile_seq` is a vector of single charaters, we can use the `c2s()` function to convert it to a string.

In [20]:
dengueseq_string <- c2s(dengueseq_fromfile_seq)
dengueseq_string

[1] "agttgttagtctacgtggaccgacaagaacagtttcgaatcggaagcttgcttaacgtagttctaacagttttttattagagagcagatctctgatgaacaaccaacggaaaaagacgggtcgaccgtctttcaatatgctgaaacgcgcgagaaaccgcgtgtcaactgtttcacagttggcgaagagattctcaaaaggattgctttcaggccaaggacccatgaaattggtgatggcttttatagcattcctaagatttctagccatacctccaacagcaggaattttggctagatggggctcattcaagaagaatggagcgatcaaagtgttacggggtttcaagaaagaaatctcaaacatgttgaacataatgaacaggaggaaaagatctgtgaccatgctcctcatgctgctgcccacagccctggcgttccatctgaccacccgagggggagagccgcacatgatagttagcaagcaggaaagaggaaaatcacttttgtttaagacctctgcaggtgtcaacatgtgcacccttattgcaatggatttgggagagttatgtgaggacacaatgacctacaaatgcccccggatcactgagacggaaccagatgacgttgactgttggtgcaatgccacggagacatgggtgacctatggaacatgttctcaaactggtgaacaccgacgagacaaacgttccgtcgcactggcaccacacgtagggcttggtctagaaacaagaaccgaaacgtggatgtcctctgaaggcgcttggaaacaaatacaaaaagtggagacctgggctctgagacacccaggattcacggtgatagccctttttctagcacatgccataggaacatccatcacccagaaagggatcatttttattttgctgatgctggtaactccatccatggccatgcggtgcgtgggaataggcaacagagacttcgtggaaggactgtcaggagctacgtgggtggatgtggtactggagcatggaagttgcgtcactaccatggcaaaagacaaaccaacactggacattgaactcttgaagacggaggtcacaaaccctgccgtcctgcgcaaactgtgcattgaagctaaaatatcaaacaccaccaccgattcgagatgtccaacacaaggagaagccacgctggtggaagaacaggacacgaactttgtgtgtcgacgaacgttcgtggacagaggctggggcaatggttgtgggctattcggaaaaggtagcttaataacgtgtgctaagtttaagtgtgtgacaaaactggaaggaaagatagtccaatatgaaaacttaaaatattcagtgatagtcaccgtacacactggagaccagcaccaagttggaaatgagaccacagaacatggaacaactgcaaccataacacctcaagctcccacgtcggaaatacagctgacagactacggagctctaacattggattgttcacctagaacagggctagactttaatgagatggtgttgttgacaatgaaaaaaaaatcatggctcgtccacaaacaatggtttctagacttaccactgccttggacctcgggggcttcaacatcccaagagacttggaatagacaagacttgctggtcacatttaagacagctcatgcaaaaaagcaggaagtagtcgtactaggatcacaagaaggagcaatgcacactgcgttgactggagcgacagaaatccaaacgtctggaacgacaacaatttttgcaggacacctgaaatgcagattaaaaatggataaactgattttaaaagggatgtcatatgtaatgtgcacagggtcattcaagttagagaaggaagtggctgagacccagcatggaactgttctagtgcaggttaaatacgaaggaacagatgcaccatgcaagatccccttctcgtcccaagatgagaagggagtaacccagaatgggagattgataacagccaaccccatagtcactgacaaagaaaaaccagtcaacattgaagcggagccaccttttggtgagagctacattgtggtaggagcaggtgaaaaagctttgaaactaagctggttcaagaagggaagcagtatagggaaaatgtttgaagcaactgcccgtggagcacgaaggatggccatcctgggagacactgcatgggacttcggttctataggaggggtgttcacgtctgtgggaaaactgatacaccagatttttgggactgcgtatggagttttgttcagcggtgtttcttggaccatgaagataggaatagggattctgctgacatggctaggattaaactcaaggagcacgtccctttcaatgacgtgtatcgcagttggcatggtcacactgtacctaggagtcatggttcaggcggactcgggatgtgtaatcaactggaaaggcagagaactcaaatgtggaagcggcatttttgtcaccaatgaagtccacacctggacagagcaatataaattccaggccgactcccctaagagactatcagcggccattgggaaggcatgggaggagggtgtgtgtggaattcgatcagccactcgtctcgagaacatcatgtggaagcaaatatcaaatgaattaaaccacatcttacttgaaaatgacatgaaatttacagtggtcgtaggagacgttagtggaatcttggcccaaggaaagaaaatgattaggccacaacccatggaacacaaatactcgtggaaaagctggggaaaagccaaaatcataggagcagatgtacagaataccaccttcatcatcgacggcccaaacaccccagaatgccctgataaccaaagagcatggaacatttgggaagttgaagactatggatttggaattttcacgacaaacatatggttgaaattgcgtgactcctacactcaagtgtgtgaccaccggctaatgtcagctgccatcaaggatagcaaagcagtccatgctgacatggggtactggatagaaagtgaaaagaacgagacttggaagttggcaagagcctccttcatagaagttaagacatgcatctggccaaaatcccacactctatggagcaatggagtcctggaaagtgagatgataatcccaaagatatatggaggaccaatatctcagcacaactacagaccaggatatttcacacaaacagcagggccgtggcacttgggcaagttagaactagattttgatttatgtgaaggtaccactgttgttgtggatgaacattgtggaaatcgaggaccatctcttagaaccacaacagtcacaggaaagacaatccatgaatggtgctgtagatcttgcacgttaccccccctacgtttcaaaggagaagacgggtgctggtacggcatggaaatcagaccagtcaaggagaaggaagagaacctagttaagtcaatggtctctgcagggtcaggagaagtggacagtttttcactaggactgctatgcatatcaataatgatcgaagaggtaatgagatccagatggagcagaaaaatgctgatgactggaacattggctgtgttcctccttctcacaatgggacaattgacatggaatgatctgatcaggctatgtatcatggttggagccaacgcttcagacaagatggggatgggaacaacgtacctagctttgatggccactttcagaatgagaccaatgttcgcagtcgggctactgtttcgcagattaacatctagagaagttcttcttcttacagttggattgagtctggtggcatctgtagaactaccaaattccttagaggagctaggggatggacttgcaatgggcatcatgatgttgaaattactgactgattttcagtcacatcagctatgggctaccttgctgtctttaacatttgtcaaaacaactttttcattgcactatgcatggaagacaatggctatgatactgtcaa

In [21]:
library(Biostrings)
matchPattern('atg',dengueseq_string)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attac

Views on a 10735-letter BString subject
subject: agttgttagtctacgtggaccgacaagaacagtt...aaaatggaatggtgctgttgaatcaacaggttct
views:
        start   end width
    [1]    95    97     3 [atg]
    [2]   137   139     3 [atg]
    [3]   224   226     3 [atg]
    [4]   236   238     3 [atg]
    [5]   298   300     3 [atg]
    ...   ...   ...   ... ...
  [288] 10455 10457     3 [atg]
  [289] 10482 10484     3 [atg]
  [290] 10497 10499     3 [atg]
  [291] 10705 10707     3 [atg]
  [292] 10710 10712     3 [atg]

From the result in `dengue_table_3bp[['atg]]` we know there are 292 occurrences of the string 'atg'. This consistent with the results returned by the `matchPattern` function.

Remember to "Save Notebook" before doing "Close and Shutdown Notebook". When you finish using JupyterHub, remember to click "File"->"Hub Control Panel", then "Stop My Server".

### Exercises

The accession number for the genome sequence of the Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is "MN908947" in the database "nuccore".

Q1. What is the GC content of this genome?  How many times does the DNA words ATG occur in the genome? How many times do stop codon sequences occur in the genome?

Q2. The `comp` function in the `seqinr` package can compute the complement of a nucleotide sequence.  How many times does each of A, C, G, T occur in the complement of the SARS-CoV-2 genome?  How do the numbers compare to the numbers in the original sequence?

Q3. The positions 28274 to 29533 of the genome encodes a nucleocapsid phosphoprotein. Use the `translate` function in the `seqinr` package to get the amino acid sequence of this protein. Since there are multiple packages that provide the `translate` function, you might want to specify that you want the one in the `seqinr` pacakge, i.e., `seqinr::translate`.

In [22]:
dengueseq <- getncbiseq("nuccore","MN908947")

In [23]:
GC(dengueseq)

[1] 0.3797278

In [30]:
dengue_table_3bp <- count(dengueseq,3)

In [34]:
names(dengue_table_3bp)

[1] "aaa" "aac" "aag" "aat" "aca" "acc" "acg" "act" "aga" "agc" "agg" "agt"
[13] "ata" "atc" "atg" "att" "caa" "cac" "cag" "cat" "cca" "ccc" "ccg" "cct"
[25] "cga" "cgc" "cgg" "cgt" "cta" "ctc" "ctg" "ctt" "gaa" "gac" "gag" "gat"
[37] "gca" "gcc" "gcg" "gct" "gga" "ggc" "ggg" "ggt" "gta" "gtc" "gtg" "gtt"
[49] "taa" "tac" "tag" "tat" "tca" "tcc" "tcg" "tct" "tga" "tgc" "tgg" "tgt"
[61] "tta" "ttc" "ttg" "ttt"